# Introduction to Python and Natural Language Technologies

__Lecture 8-2, Data preparation__

__March 30, 2021__

__Judit Ács__

This is a helper notebook that downloads and prepares the dataset used in Lecture 8.

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess

In [2]:
unimorph_path = "data/unimorph_repos/hun"
os.makedirs("data", exist_ok=True)
os.makedirs("data/unimorph_repos", exist_ok=True)

In [3]:
pipe = subprocess.Popen(f"git clone git@github.com:unimorph/hun.git {unimorph_path}",
                        shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
stdout, stderr = pipe.communicate()

In [4]:
stdout

b''

In [5]:
stderr

b"fatal: destination path 'data/unimorph_repos/hun' already exists and is not an empty directory.\n"

In [6]:
hun = pd.read_table(f"{unimorph_path}/hun", names=['lemma', 'infl', 'tags'], skip_blank_lines=True)
hun.head()

,lemma,infl,tags
0,gravitáció,gravitáción,N;ON+ESS;SG
1,gravitáció,gravitációként,N;FRML;SG
2,gravitáció,gravitációkban,N;IN+ESS;PL
3,gravitáció,gravitációkhoz,N;AT+ALL;PL
4,gravitáció,gravitáció,N;NOM;SG


In [7]:
hun['pos'] = hun.tags.str[0]

In [8]:
hun[hun.tags.str[0]=='V'].sample(5)

,lemma,infl,tags,pos
146230,megdönt,megdöntöttünk,V;IND;PST;INDF;1;PL,V
485702,elvág,elvágnának,V;COND;PRS;INDF;3;PL,V
311457,transzformál,transzformálnotok,V;NFIN;2;PL,V
248036,sodródik,sodródjon,V;SBJV;PRS;INDF;3;SG,V
338166,ventilál,ventilálták,V;IND;PST;DEF;3;PL,V


In [9]:
def get_case(tags):
    if tags[0] == 'N':
        return tags.split(';')[1]
    return None

hun['case'] = hun['tags'].apply(get_case)
hun = hun[hun.pos=='N']

In [10]:
print(hun.case.nunique())
hun.case.value_counts(dropna=False)

17


PRP       25260
NOM       25260
ON+ABL    25260
ON+ESS    25260
IN+ALL    25260
INST      25260
TRANS     25260
AT+ESS    25260
IN+ABL    25260
AT+ALL    25260
DAT       25260
ON+ALL    25260
ACC       25260
TERM      25260
AT+ABL    25253
IN+ESS    25252
FRML      25248
Name: case, dtype: int64

In [11]:
lemmas = hun.lemma.unique()
len(lemmas), type(lemmas), len(hun)

(12782, numpy.ndarray, 429393)

In [12]:
np.random.seed(12)
np.random.shuffle(lemmas)
train_size = int(0.8 * len(lemmas))
dev_size = int(0.1 * len(lemmas))
train_lemmas = lemmas[:train_size]
dev_lemmas = lemmas[train_size:train_size+dev_size]
test_lemmas = lemmas[train_size+dev_size:]

train_lemmas = set(train_lemmas)
dev_lemmas = set(dev_lemmas)
test_lemmas = set(test_lemmas)

In [13]:
len(train_lemmas & dev_lemmas), len(train_lemmas & test_lemmas), len(test_lemmas & dev_lemmas)

(0, 0, 0)

In [14]:
hun_train = hun[hun.lemma.isin(train_lemmas)]
hun_dev = hun[hun.lemma.isin(dev_lemmas)]
hun_test = hun[hun.lemma.isin(test_lemmas)]
len(hun_train), len(hun_dev), len(hun_test)

(343484, 42752, 43157)

In [15]:
os.makedirs("data", exist_ok=True)
os.makedirs("data/unimorph", exist_ok=True)

In [16]:
hun_train.to_csv("data/unimorph/hun_train.tsv", sep="\t", index=False)
hun_dev.to_csv("data/unimorph/hun_dev.tsv", sep="\t", index=False)
hun_test.to_csv("data/unimorph/hun_test.tsv", sep="\t", index=False)

In [17]:
hun_train.head()

,lemma,infl,tags,pos,case
0,gravitáció,gravitáción,N;ON+ESS;SG,N,ON+ESS
1,gravitáció,gravitációként,N;FRML;SG,N,FRML
2,gravitáció,gravitációkban,N;IN+ESS;PL,N,IN+ESS
3,gravitáció,gravitációkhoz,N;AT+ALL;PL,N,AT+ALL
4,gravitáció,gravitáció,N;NOM;SG,N,NOM
